In [29]:
import numpy as np
#from strip import LEDPixel, Strip
from enum import Enum
import random

#TODO: move
PixelRecordDType = [
    ("i", np.uint8),
    ("r", np.uint8),
    ("g", np.uint8),
    ("b", np.uint8),
]

PixelRecordDType = [
	("i", np.uint8),
	("c", 
		[("r", np.uint8),
		("g", np.uint8),
		("b", np.uint8)])
]

def makeStrip(length:int):
    return np.zeros(shape=length, dtype=PixelRecordDType).view(np.recarray)


class Modes(Enum):
    off = 0
    solid = 1
    alternate = 2
    gradient_scroll = 3
    random_strobe = 4  #Chunks of size, at rate
    rainbow = 5

class Align(Enum):
    left = 0
    center = 1
    right = 2

In [31]:
st = makeStrip(10)

st.i = 0xe1
st[0] = 0xff
st[1].c.r = 13
bytes(st)

b'\xff\xff\xff\xff\xe1\r\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00\xe1\x00\x00\x00'

In [34]:
b = st[1:4]
b.c=(1,2,3)
st

rec.array([(255, (255, 255, 255)), (225, (  1,   2,   3)),
           (  0, (  1,   2,   3)), (225, (  1,   2,   3)),
           (225, (  0,   0,   0)), (225, (  0,   0,   0)),
           (225, (  0,   0,   0)), (225, (  0,   0,   0)),
           (225, (  0,   0,   0)), (225, (  0,   0,   0))],
          dtype=[('i', 'u1'), ('c', [('r', 'u1'), ('g', 'u1'), ('b', 'u1')])])

In [50]:
np.array(range(20))[1:5]

array([1, 2, 3, 4])

'02468'

In [2]:

class Color(): #8bit color math! All values live as 8 bit RGB
    def __init__(self, *args, **kwargs):#r:int, g:int, b:int):
        if len(args)==3:
            self.r = args[0]
            self.g = args[1]
            self.b = args[2]
        elif len(args)==1 and isinstance(args[0], str): #"WEB" mode
            self.r = int(args[0][1:3], 16)
            self.g = int(args[0][3:5], 16)
            self.b = int(args[0][5:7], 16)
        elif (("h" in kwargs) and ("s" in kwargs) and ("v" in kwargs)):
            self.r=255
            self.g=255
            self.b=255
            pass #TODO rgb from HSV?
        elif (("r" in kwargs) and ("g" in kwargs) and ("b" in kwargs)):
            self.r = kwargs["r"]
            self.g = kwargs["g"]
            self.b = kwargs["b"]
        else:
            self.r=255
            self.g=60
            self.b=60

    @property
    def rgb(self):
        return (self.r,self.g,self.b)
    @rgb.setter
    def rgb(self,rgb):
        self.r, self.g, self.b = rgb

    def range_to(self, color2:Color, length:int):
        return [self]*length


In [3]:

def control(strip, c1:Color, c2:Color, intensity:int, mode:Modes, parameter1:int, parameter2:int, parameter3:int, startIndex:int, stopIndex:int, align:Align):
    #c1 = Color(color1)
    #c2 = Color(color2)

    #Base parameters
    width = parameter1*3 + 1
    widthScale = 256.0/width
    size = parameter2
    shift = parameter3-128 #Its signed

    [p.setIntensity5bit(0) for p in strip[:startIndex]]
    [p.setIntensity5bit(intensity) for p in strip[startIndex:stopIndex+1]]
    [p.setIntensity5bit(0) for p in strip[stopIndex+1:]]

    
    # Handle alignment modes
    if align == Align.left:
        livePix = enumerate(strip[startIndex:stopIndex+1])
    elif align == Align.center:
        halfStripWidth = (stopIndex - startIndex)//2
        stripCenter = startIndex + halfStripWidth
        i = abs(stripCenter-i)
        livePix = enumerate(strip) #TODO
    elif align == Align.right:
        livePix = enumerate(strip[stopIndex:startIndex-1:-1])
    else: 
        livePix = []
    
    if mode in (Modes.off, Modes.solid):
        if mode == Modes.off:
            rgb = (0,0,0)
        else:
            rgb = c1.rgb
        [p.setRGB8bit(rgb) for i,p in livePix]

    elif mode in (Modes.gradient_scroll, Modes.rainbow):
        if mode == Modes.gradient_scroll:
            gradient = list(c1.range_to(c2, 128))
            gradient += gradient[::-1] #Mirror the list!
        elif mode == Modes.rainbow:
            gradient = [Color(hue=theta, saturation=1, luminance=0.5) for theta in np.linspace(0.,1., 256, endpoint=False)]

        [p.setRGB8bit( gradient[ (int((i%width)*widthScale)+shift-size)%256 ].rgb ) for i,p in livePix]

    elif mode == Modes.alternate:
        rgb1 = c1.rgb
        rgb2 = c2.rgb
        upper_cutoff = (size*2)
        lower_cutoff = (size*2-256)
        [p.setRGB8bit( rgb1 if (lower_cutoff <= (int((i%width)*widthScale)+shift-size)%256 < upper_cutoff) else rgb2 ) for i,p in livePix]

    elif mode == Modes.random_strobe:
        pass




In [ ]:

def dmx(universe_data, patch):
    #print(universe_data)
    for address, st in patch:
        data = universe_data[address-1:address+14]
        if (data[7] >= 60) or (data[0] >= 96):
            return None ##There was an error, probably bc of previs software trying to be secure
        mode = Modes(data[7] // 10)
        align = Align(data[0] // 32)

        control(st, 
                Color(data[1],data[2],data[3]),
                Color(data[4],data[5],data[6]),
                data[0]%32,
                mode,
                data[8],
                data[9],
                data[10],
                data[11]*256+data[12],
                data[13]*256+data[14],
                align
                )


In [30]:
from dmxToLED import StripHW as Strip
import numpy as np

shift = 5
size = 64
width = 20

strip = Strip(100)
def z(width, size, shift):
    widthScale = 256.0/width
    
    lower_cutoff = int(-shift/widthScale) % width
    upper_cutoff = int((size*2-shift)/widthScale) % width
    invert = lower_cutoff > upper_cutoff
    if invert:
        lower_cutoff, upper_cutoff = upper_cutoff, lower_cutoff
    if invert ^ (size>=128):
        rgb1 = (99,99,99)
        rgb2 = (0,0,0)
    else:
        rgb1 = (0,0,0)
        rgb2 = (99,99,99)
    print(f"Cutoffs: {lower_cutoff}, {upper_cutoff};  C1={rgb1}")
    #for subStrip in np.split(strip, range(width, len(strip)+width-1, width)):
    #    subStrip[:lower_cutoff].c = rgb1
    #    subStrip[lower_cutoff:upper_cutoff].c = rgb2
    #    subStrip[upper_cutoff:].c = rgb1
    return strip

for size in range(0,256,16):
    z(width, size, shift)
#
#strip

Cutoffs: 0, 0;  C1=(0, 0, 0)
Cutoffs: 0, 2;  C1=(0, 0, 0)
Cutoffs: 0, 4;  C1=(0, 0, 0)
Cutoffs: 0, 7;  C1=(0, 0, 0)
Cutoffs: 0, 9;  C1=(0, 0, 0)
Cutoffs: 0, 12;  C1=(0, 0, 0)
Cutoffs: 0, 14;  C1=(0, 0, 0)
Cutoffs: 0, 17;  C1=(0, 0, 0)
Cutoffs: 0, 19;  C1=(99, 99, 99)
Cutoffs: 0, 2;  C1=(99, 99, 99)
Cutoffs: 0, 4;  C1=(99, 99, 99)
Cutoffs: 0, 7;  C1=(99, 99, 99)
Cutoffs: 0, 9;  C1=(99, 99, 99)
Cutoffs: 0, 12;  C1=(99, 99, 99)
Cutoffs: 0, 14;  C1=(99, 99, 99)
Cutoffs: 0, 17;  C1=(99, 99, 99)


In [7]:
strip[1:2].c

rec.array([(0, 0, 0)],
          dtype=[('r', 'u1'), ('g', 'u1'), ('b', 'u1')])

In [55]:
import numpy as np

ColorRecordDType = [
	("b", np.uint8),
	("g", np.uint8),
	("r", np.uint8)]


def mirror(a):
    return np.concatenate( (a, np.flip(a, axis=0)), axis=0)



z = mirror(np.linspace( (1,1,3), (10,19,30), 4, axis=0)).astype(int)

#np.array([( 1,  1,  3),
#          ( 4,  7, 12),
#          ( 7, 13, 21),
#          (10, 19, 30),
#          (10, 19, 30),
#          ( 7, 13, 21),
#          ( 4,  7, 12),
#          ( 1,  1,  3)], dtype=ColorRecordDType)

#z.astype(ColorRecordDType)
np.linspace( (1,1,3), (10,19,30), 10, axis=0).astype(np.uint8).view(ColorRecordDType)[:,0]

array([( 1,  1,  3), ( 2,  3,  6), ( 3,  5,  9), ( 4,  7, 12),
       ( 5,  9, 15), ( 6, 11, 18), ( 7, 13, 21), ( 8, 15, 24),
       ( 9, 17, 27), (10, 19, 30)],
      dtype=[('b', 'u1'), ('g', 'u1'), ('r', 'u1')])

In [68]:
def HSV_2_RGB(H, S, V):
    ''' Converts an integer HSV tuple (value range from 0 to 255) to an RGB tuple '''

    #https://stackoverflow.com/questions/24152553/hsv-to-rgb-and-back-without-floating-point-math-in-python

    # Check if the color is Grayscale
    if S == 0:
        return (V,V,V)

    # Make hue 0-5
    region = H // 43

    # Find remainder part, make it from 0-255
    #remainder = (H - (region * 43)) * 6
    remainder = (H%43) * 6

    # Calculate temp vars, doing integer multiplication
    P = (V * (255 - S)) >> 8
    Q = (V * (255 - ((S * remainder) >> 8))) >> 8
    T = (V * (255 - ((S * (255 - remainder)) >> 8))) >> 8

    # Assign temp vars based on color cone region
    return ( #RGB, 8bit
        (V,T,P),
        (Q,V,P),
        (P,V,T),
        (P,V,Q),
        (T,P,V),
        (V,P,Q)
    )[region]


def NP_HSV_TO_BGR(array:np.ndarray) -> np.recarray[ColorRecordDType]:
    H = array[:,0]
    S = array[:,1]
    V = array[:,2]

    region = H//43
    remainder = (H%43) * 6

    P = (V * (255 - S)) >> 8
    Q = (V * (255 - ((S * remainder) >> 8))) >> 8
    T = (V * (255 - ((S * (255 - remainder)) >> 8))) >> 8

    lookup = np.array([ #BGR, 8bit
        (P,T,V),
        (P,V,Q),
        (T,V,P),
        (Q,V,P),
        (V,P,T),
        (Q,P,V)
    ])

    return lookup[region,...,range(len(array))]

print(NP_HSV_TO_BGR(np.array([[1,255,255],[44,255,255]])))


H,S,V:
[ 1 44]
[255 255]
[255 255]
P,Q,T:
[0 0]
[249 249]
[6 6]
region:
[0 1]
lookup:
[[[  0   0]
  [  6   6]
  [255 255]]

 [[  0   0]
  [255 255]
  [249 249]]

 [[  6   6]
  [255 255]
  [  0   0]]

 [[249 249]
  [255 255]
  [  0   0]]

 [[255 255]
  [  0   0]
  [  6   6]]

 [[249 249]
  [  0   0]
  [255 255]]]
[[  0   6 255]
 [  0 255 249]]
(249, 255, 0)


In [80]:
from color import Color
def RGB_2_HSV(RGB):
    ''' Converts an integer RGB tuple (value range from 0 to 255) to an HSV tuple '''

    # Unpack the tuple for readability
    R, G, B = RGB

    # Compute the H value by finding the maximum of the RGB values
    RGB_Max = max(RGB)
    RGB_Min = min(RGB)

    # Compute the value
    V = RGB_Max
    if V == 0:
        H = S = 0
        return (H,S,V)


    # Compute the saturation value
    S = 255 * (RGB_Max - RGB_Min) // V

    if S == 0:
        H = 0
        return (H, S, V)

    # Compute the Hue
    if RGB_Max == R:
        H = 0 + 43*(G - B)//(RGB_Max - RGB_Min)
    elif RGB_Max == G:
        H = 85 + 43*(B - R)//(RGB_Max - RGB_Min)
    else: # RGB_MAX == B
        H = 171 + 43*(R - G)//(RGB_Max - RGB_Min)

    return (H, S, V)

def HSV_2_RGB(H, S, V):
    ''' Converts an integer HSV tuple (value range from 0 to 255) to an RGB tuple '''

    #https://stackoverflow.com/questions/24152553/hsv-to-rgb-and-back-without-floating-point-math-in-python

    # Check if the color is Grayscale
    if S == 0:
        return (V,V,V)

    # Make hue 0-5
    region = H // 43

    # Find remainder part, make it from 0-255
    #remainder = (H - (region * 43)) * 6
    remainder = (H%43) * 6

    # Calculate temp vars, doing integer multiplication
    P = (V * (255 - S)) >> 8
    Q = (V * (255 - ((S * remainder) >> 8))) >> 8
    T = (V * (255 - ((S * (255 - remainder)) >> 8))) >> 8

    # Assign temp vars based on color cone region
    return ( #RGB, 8bit
        (V,T,P),
        (Q,V,P),
        (P,V,T),
        (P,V,Q),
        (T,P,V),
        (V,P,Q)
    )[region]



def NP_HSV_TO_BGR_ColorRecordDType(array:np.ndarray) -> np.recarray[ColorRecordDType]:
    H = array[...,0]
    S = array[...,1]
    V = array[...,2]

    region = H//43
    remainder = (H%43) * 6

    print("S")
    print(S)
    print("remainder")
    print(remainder)
    print("S*remainder")
    print((S*remainder))

    P = (V * (255 - S))
    Q = (V * (255 - (S * remainder)))
    T = (V * (255 - (S * (255 - remainder))))

    lookup = np.array([ #BGR, 8bit
        (P,T,V),
        (P,V,Q),
        (T,V,P),
        (Q,V,P),
        (V,P,T),
        (Q,P,V)
    ])


    print(region)
    print(lookup)

    return lookup[region,...,range(len(array))].view(ColorRecordDType)[:,0]

def np_range_rainbow(self, length:int) -> np.recarray[ColorRecordDType]:
    hsvspace = np.linspace( (0,255,255) , (255,255,255), length)
    return NP_HSV_TO_BGR_ColorRecordDType(hsvspace)


def np_range_to_hsv(self, color2, length:int) -> np.recarray[ColorRecordDType]:
    #TODO linspace should wrap around for hue!
    hsv1 = RGB_2_HSV(self.rgb)
    hsv2 = RGB_2_HSV(color2.rgb)
    hsvspace = np.linspace( hsv1, hsv2, length, axis=0).astype(np.uint8)
    #print(hsvspace)
    
    return NP_HSV_TO_BGR_ColorRecordDType(hsvspace)


np_range_to_hsv(Color(r=255, g=0,b=0), Color(r=0, g=255,b=0), 10)


S
[255 255 255 255 255 255 255 255 255 255]
remainder
[  0  54 108 168 222  24  78 138 192 252]
S*remainder
[  0 202 148  88  34 232 178 118  64   4]
[0 0 0 0 0 1 1 1 1 1]
[[[  0   0   0   0   0   0   0   0   0   0]
  [  2  56 110 170 224  26  80 140 194 254]
  [255 255 255 255 255 255 255 255 255 255]]

 [[  0   0   0   0   0   0   0   0   0   0]
  [255 255 255 255 255 255 255 255 255 255]
  [  1 203 149  89  35 233 179 119  65   5]]

 [[  2  56 110 170 224  26  80 140 194 254]
  [255 255 255 255 255 255 255 255 255 255]
  [  0   0   0   0   0   0   0   0   0   0]]

 [[  1 203 149  89  35 233 179 119  65   5]
  [255 255 255 255 255 255 255 255 255 255]
  [  0   0   0   0   0   0   0   0   0   0]]

 [[255 255 255 255 255 255 255 255 255 255]
  [  0   0   0   0   0   0   0   0   0   0]
  [  2  56 110 170 224  26  80 140 194 254]]

 [[  1 203 149  89  35 233 179 119  65   5]
  [  0   0   0   0   0   0   0   0   0   0]
  [255 255 255 255 255 255 255 255 255 255]]]


array([(0,   2, 255), (0,  56, 255), (0, 110, 255), (0, 170, 255),
       (0, 224, 255), (0, 255, 233), (0, 255, 179), (0, 255, 119),
       (0, 255,  65), (0, 255,   5)],
      dtype=[('b', 'u1'), ('g', 'u1'), ('r', 'u1')])